In [135]:
import numpy as np
import qutip as qt

In [136]:
ket_0 = qt.basis(2, 0)
ket_1 = qt.basis(2, 1)

ket_00 = qt.tensor(ket_0, ket_0)
ket_11 = qt.tensor(ket_1, ket_1)
ket_01 = qt.tensor(ket_0, ket_1)
ket_10 = qt.tensor(ket_1, ket_0)

a = (np.sqrt(5) - 1) / 2

ket_phi = a * (ket_01 + ket_10) + np.sqrt(1-2*a**2) * ket_11
ket_phi

Quantum object: dims=[[2, 2], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.        ]
 [0.61803399]
 [0.61803399]
 [0.48586827]]

In [137]:
rho_phi = ket_phi * ket_phi.dag()
rho_phi

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.         0.         0.         0.        ]
 [0.         0.38196601 0.38196601 0.30028311]
 [0.         0.38196601 0.38196601 0.30028311]
 [0.         0.30028311 0.30028311 0.23606798]]

In [138]:
proj_0 = ket_0 * ket_0.dag()
proj_1 = ket_1 * ket_1.dag()

psi = 1 / np.sqrt(1-a**2) * (np.sqrt(1-2*a**2) * ket_0 - a * ket_1)
proj_psi = psi * psi.dag()

proj_0, proj_1, proj_psi, 1 - proj_psi

(Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
 Qobj data =
 [[1. 0.]
  [0. 0.]],
 Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
 Qobj data =
 [[0. 0.]
  [0. 1.]],
 Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
 Qobj data =
 [[ 0.38196601 -0.48586827]
  [-0.48586827  0.61803399]],
 Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
 Qobj data =
 [[0.61803399 0.48586827]
  [0.48586827 0.38196601]])

# 1 Copy

In [139]:
def p(a, b, x, y):
    if x == 1:
        proj_a = proj_0 if a == 0 else proj_1
    else:
        proj_a = proj_psi if a == 0 else (1 - proj_psi)

    if y == 1:
        proj_b = proj_0 if b == 0 else proj_1
    else:
        proj_b = proj_psi if b == 0 else (1 - proj_psi)

    proj_ab = qt.tensor(proj_a, proj_b)
    return (proj_ab * rho_phi).tr()

p(0, 1, 0, 1), p(1,0,1,0), p(0,0,1,1), p(0,0,0,0)

(0.0, 0.0, 0.0, (0.09016994374947418+0j))

Note - the above has the definition of x swapped, so that when x=0, we project in the psi basis, and when x=1, we project in the logical basis

# 2 Copies

In [140]:
def phi_n(n):
    return qt.tensor([ket_phi] * n)

phi_2 = phi_n(2)
phi_2

Quantum object: dims=[[2, 2, 2, 2], [1]], shape=(16, 1), type='ket', dtype=Dense
Qobj data =
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.38196601]
 [0.38196601]
 [0.30028311]
 [0.        ]
 [0.38196601]
 [0.38196601]
 [0.30028311]
 [0.        ]
 [0.30028311]
 [0.30028311]
 [0.23606798]]

In [141]:
def proj_ab_n(a_vals, b_vals, x, y):
    n = len(a_vals)
    projs_a = []
    projs_b = []
    for i in range(n):
        a = a_vals[i]
        b = b_vals[i]

        if x == 1:
            proj_a = proj_0 if a == 0 else proj_1
        else:
            proj_a = proj_psi if a == 0 else (1 - proj_psi)
        projs_a.append(proj_a)

        if y == 1:
            proj_b = proj_0 if b == 0 else proj_1
        else:
            proj_b = proj_psi if b == 0 else (1 - proj_psi)
        projs_b.append(proj_b)

    return qt.tensor([qt.tensor(projs_a[i], projs_b[i]) for i in range(n)])


def p_n(a_vals, b_vals, x, y):
    n = len(a_vals)
    assert len(b_vals) == n

    proj_ab = proj_ab_n(a_vals, b_vals, x, y)
    rho = phi_n(n) * phi_n(n).dag()
    return (proj_ab * rho).tr()


In [142]:
# (a1 b1)(a2 b2) ... (an bn)
# (00)(00) + (00)(01) + (00)(10) + (00)(11) + (01)(00) + (10)(00) + (11)(00)

# (a1 a2) (b1 b2)
# (00)(00) + (00)(01) + (01)(00) + (01)(01) + (00)(10) + (10)(00) + (10)(10)

np.sum([p_n([0,0], [0,0], 0, 0),
        p_n([0,0], [0,1], 0, 0),
        p_n([0,1], [0,0], 0, 0),
        p_n([0,1], [0,1], 0, 0),
        p_n([0,0], [1,0], 0, 0),
        p_n([1,0], [0,0], 0, 0),
        p_n([1,0], [1,0], 0, 0)])

np.complex128(0.1722092687431651+0j)

# N Copies

In [143]:
# Pairs of bits, times n, iff at least one pair of bits is (0,0)
from itertools import product

def get_sum_terms(n: int):
    pairs = [(0,0), (0,1), (1,0), (1,1)]
    all_terms = []
    for seq in product(pairs, repeat=n):
        if (0,0) in seq:
            all_terms.append(list(seq))  # convert tuple to list
    return np.array([list(list(t) for t in term) for term in all_terms])

get_sum_terms(3)

array([[[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 1]],

       [[0, 0],
        [0, 0],
        [1, 0]],

       [[0, 0],
        [0, 0],
        [1, 1]],

       [[0, 0],
        [0, 1],
        [0, 0]],

       [[0, 0],
        [0, 1],
        [0, 1]],

       [[0, 0],
        [0, 1],
        [1, 0]],

       [[0, 0],
        [0, 1],
        [1, 1]],

       [[0, 0],
        [1, 0],
        [0, 0]],

       [[0, 0],
        [1, 0],
        [0, 1]],

       [[0, 0],
        [1, 0],
        [1, 0]],

       [[0, 0],
        [1, 0],
        [1, 1]],

       [[0, 0],
        [1, 1],
        [0, 0]],

       [[0, 0],
        [1, 1],
        [0, 1]],

       [[0, 0],
        [1, 1],
        [1, 0]],

       [[0, 0],
        [1, 1],
        [1, 1]],

       [[0, 1],
        [0, 0],
        [0, 0]],

       [[0, 1],
        [0, 0],
        [0, 1]],

       [[0, 1],
        [0, 0],
        [1, 0]],

       [[0, 1],
        [0, 0],
        [1, 1]],



In [144]:
np.sum([p_n(term.transpose()[0], term.transpose()[1], 0, 0) for term in get_sum_terms(1)])

np.complex128(0.09016994374947418+0j)

# k Outcomes

In [211]:
def p_n_k(n):
    if n == 1: 
        return # Not supported
    terms = get_sum_terms(n)
    k = len(terms)

    a_vals_k = terms.transpose()[0].transpose()
    b_vals_k = terms.transpose()[1].transpose()

    proj_ab_n_sum = 0

    for i in range(k):
        a_vals = a_vals_k[i]
        b_vals = b_vals_k[i]
        proj_ab_n_i = proj_ab_n(a_vals, b_vals, 0, 0)
        proj_ab_n_sum += proj_ab_n_i

    rho_n = phi_n(n) * phi_n(n).dag()
    p = (proj_ab_n_sum * rho_n).tr()
    return p

n=5
p_n_k(n)

(0.3765503316023426+0j)

In [215]:
for i in range(2, 6):
    print(i, p_n_k(i))

2 (0.1722092687431651+0j)
3 (0.2468511124169302+0j)
4 (0.31476250524527366+0j)
5 (0.3765503316023426+0j)
